# Simuleer een epidemie



<div class="alert alert-box alert-success">
In dit project zullen we bestuderen hoe ziektes zich kunnen verspreiden doorheen een (sociaal) netwerk. We zullen onderzoeken hoe de structuur van een netwerk een invloed kan hebben op hoe snel een ziekte doorgegeven wordt. Finaal zullen we ook verschillende strategieën bekijken om de verspreiding van een ziekte tegen te gaan.
</div>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from scipy.spatial import distance_matrix

## Het SIR-model

Een van de eenvoudigste manieren om ziekteverspreiding in een gemeenschap te modelleren, is aan de hand van het SIR-model. **SIR staat voor *Susceptible* (vatbaar), *Infected* (geïnfecteerd) en *Resistant* (resistent of hersteld), de drie types individuen die in een gemeenschap voorkomen.** <br>
Het SIR-model bestaat uit drie vergelijkingen die de veranderingen van het aantal individuen in een bepaalde groep beschrijven. De variabelen die de toestand beschrijven zijn:

-  $S(t)$: het aantal vatbare individuen op tijdstip $t$;
-  $I(t)$: het aantal geïnfecteerde individuen op tijdstip $t$;
-  $R(t)$: het aantal resistente individuen op tijdstip $t$.

Hierbij is t de tijd in een bepaalde tijdseenheid (de tijdseenheid wordt gekozen afhankelijk van het probleem).

In deze beschrijving maken we een eerste grote vereenvoudiging van de werkelijkheid. We nemen aan dat elk van deze variabelen reële getallen zijn, en dat het aantal individuen in elke groep continu kan variëren. In werkelijkheid zijn het discrete waarden: het aantal geïnfecteerden en vatbare individuen is een natuurlijk getal, je bent immers besmet of je bent het niet. Modelleerders werken echter graag met continue variabelen omdat ze dan de technieken van de wiskundige analyse kunnen gebruiken.

> **Oefening 1**: Onder welke omstandigheden gaat deze continue benadering ongeveer op? Denk je dat je dit model kan gebruiken om een gezin van vier personen te beschrijven?

Deze drie variabelen worden aan elkaar gelinkt aan de hand van drie differentiaalvergelijkingen (die elk een verandering in de tijd beschrijven). Hierbij nemen we aan dat de grootte van de populatie ongewijzigd blijft: we nemen dus aan dat, gedurende de tijdspanne die het model beschrijft, er niemand geboren wordt en ook niemand sterft. Eigenlijk beperken we ons hier tot de verspreiding van een relatief onschuldige ziekte zoals een verkoudheid. We kunnen de situatie dus voorstellen met het volgende stelsel differentiaalvergelijkingen: 
 
$$
\frac{\text{d}S(t)}{\text{d}t} = -\beta \, S(t) \, I(t)
$$

$$
\frac{\text{d}I(t)}{\text{d}t} = \beta \, S(t) \,I(t) - \gamma \, I(t)
$$

$$
\frac{\text{d}R(t)}{\text{d}t} = \gamma \, I(t)
$$

<div class="alert alert-box alert-info">
Elke vergelijking vertelt ons hoe het aantal mensen in elke groep wijzigt doorheen de tijd. Daaruit kunnen we ook berekenen hoeveel mensen zich op een bepaald moment bevinden in elke groep. De parameters $\beta$ en $\gamma$ spelen daarbij een fundamentele rol.
</div>

De vergelijkingen zijn gekoppeld via de *overgangspercentages*. Elk overgangspercentage vertelt ons hoe van de ene naar de andere groep wordt overgegaan. <br>
Het overgangspercentage van vatbaar (S) naar geïnfecteerd (I) hangt af van het contact tussen een vatbare persoon en een geïnfecteerde persoon. We noemen dit *infectiepercentage* $\beta$. Dit betekent dat één geïnfecteerde persoon $\beta S$ personen zal besmetten. Het aantal vatbare personen vermindert dus met $\beta S I$ per tijdseenheid. <br>
Het overgangspercentage van geïnfecteerd (I) naar resistent (R) hangt alleen af van het *herstelpercentage*, dat we $\gamma$ noemen. Het aantal geïnfecteerde personen vermindert dus met  $\gamma I$ per tijdseenheid.

> **Oefening 2**: Toon aan met een berekening dat het totaal aantal individuen in de populatie $(S(t)+I(t)+R(t))$ constant zal blijven.

<div class="alert alert-box alert-info">
Het SIR-model is moeilijk om exact te worden opgelost. Dit is het geval bij veel differentiaalvergelijkingen die optreden in de biologische wetenschappen. We moeten dus een <em>numerieke benadering</em> van de oplossing vinden. Dit betekent dat we een algoritme zullen gebruiken om een geschatte maar nauwkeurige oplossing te vinden. Vanuit deze oplossing kunnen we leren hoe de verschillende variabelen in de loop van de tijd veranderen.
</div>

Er zijn verschillende mogelijkheden om dit te doen: 

-  We zouden ons continue probleem kunnen vervangen door een **discrete** tegenhanger. <br>
Dit zou ons toelaten bepaalde numerieke methoden te gebruiken om een benaderende oplossing te krijgen. 

-  Anderzijds kunnen we een **iteratieve** methode gebruiken. <br>Uitgaande van een initiële schatting, maken iteratieve methoden opeenvolgende benaderingen die stapsgewijs convergeren naar de exacte oplossing.

## Iteratieve manier

Met behulp van computers is het gemakkelijk om iteratief een numerieke oplossing voor het SIR-model te vinden. <br>
Om dit te doen, vertrekken we van een beginvoorwaarde: het is logisch om te beginnen met een populatie met nul resistente personen, een paar geïnfecteerde personen en de rest vatbaar. Vervolgens kunnen we onze numerieke oplossing gebruiken om het aantal mensen in elke groep op bepaalde tijdstippen te berekenen.

Via de Python-module SciPy kunnen we eenvoudig dergelijke differentiaalvergelijkingen simuleren. Eerst moeten we de differentiaalvergelijkingen *implementeren*: we stoppen daartoe de drie vergelijkingen hierboven gegeven in een rijmatrix.

Met behulp van de Python-module NumPy kan een matrix voorgesteld worden met een *Numpy array*.

In [ ]:
def SIR(t, y, beta, gamma):
    """Differentiaalvergelijkingen die S, I en R in functie van de tijd t bepalen."""
    S, I, R = y
    return np.array([-beta * S * I,
                    beta * S * I - gamma * I,
                    gamma * I])

Nu kunnen we het stelsel differentiaalvergelijkingen *numeriek oplossen* met de [solve_ivp](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html)-functie voor een bepaalde *beginsituatie*. <br>We beschouwen een populatie met 1000 mensen, waarvan initieel één persoon geïnfecteerd is.

In [ ]:
# voorbeeld 1 
# beginsituatie
S0 = 999
I0 = 1
R0 = 0

beta = 0.001
gamma = 0.1


oplossing = solve_ivp(SIR,                     # functie met parameters
                      [0, 100],                # tijdsinterval waarin we simuleren
                      np.array([S0, I0, R0]),  # initiële omstandigheden
                      args=(beta, gamma))    # parameters van stelsel differentiaalvergelijkingen 

Deze oplossing kunnen we dan grafisch weergeven:

In [ ]:
# voorbeeld 1 grafiek oplossing S, I, R 
plt.figure()

plt.plot(oplossing.t, oplossing.y[1], color="purple")  # I 
plt.plot(oplossing.t, oplossing.y[2], color="green")   # R
plt.plot(oplossing.t, oplossing.y[0], color="orange")  # S

plt.show()

In [ ]:
# voorbeeld 1 grafiek verdeling populatie over S, I, R in functie van de tijd
plt.figure()

plt.stackplot(oplossing.t, oplossing.y[[1,0,2],:],
              labels=["I", "S", "R"],
              colors=["red", "yellow", "lightgreen"])
plt.xlabel("Tijd")
plt.ylabel("Aantal personen")
plt.legend(loc=0)

plt.show()

In [ ]:
# grafiek voorbeeld 1 combinatie verdeling populatie en S, I, R
plt.figure()

plt.stackplot(oplossing.t, oplossing.y[[1,0,2],:],
              labels=["I", "S", "R"],
              colors=["red", "yellow", "lightgreen"])
plt.xlabel("Tijd")
plt.ylabel("Aantal personen")
plt.legend(loc=0)

plt.plot(oplossing.t, oplossing.y[1], color="purple")  # I 
plt.plot(oplossing.t, oplossing.y[2], color="green")   # R
plt.plot(oplossing.t, oplossing.y[0], color="orange")  # S

plt.show()

> **Oefening 3**: Simuleer een aantal situaties door de parameters aan te passen: 
-  Wat als initieel de helft van de populatie resistent was? 
-  Wat als initieel 80 % van de populatie resistent was?

In [ ]:
# voorbeeld 2 
# beginsituatie
S0 = 1099
I0 = 1
R0 = 0

beta = 0.0001
gamma = 0.048


oplossing = solve_ivp(SIR,                     # functie met parameters
                      [0, 365],                # tijdsinterval waarin we simuleren
                      np.array([S0, I0, R0]),  # initiële omstandigheden
                      t_eval=np.linspace(0,365,36),   # aantal punten van oplossing
                      args=(beta, gamma))    # parameters van stelsel differentiaalvergelijkingen 

In [ ]:
# voorbeeld 2 grafiek oplossing S, I, R 
plt.figure()

plt.plot(oplossing.t, oplossing.y[1], color="purple")  # I 
plt.plot(oplossing.t, oplossing.y[2], color="green")   # R
plt.plot(oplossing.t, oplossing.y[0], color="orange")  # S

plt.show()

In [ ]:
# grafiek voorbeeld 2 grafiek verdeling populatie over S, I, R in functie van de tijd
plt.figure()

plt.stackplot(oplossing.t, oplossing.y[[1,0,2],:],
              labels=["I", "S", "R"],
              colors=["red", "yellow", "green"])
plt.xlabel("Tijd")
plt.ylabel("Aantal personen")
plt.legend(loc=0)

plt.show()

In [ ]:
# grafiek voorbeeld 2 combinatie verdeling populatie en S, I, R
plt.figure()

plt.stackplot(oplossing.t, oplossing.y[[1,0,2],:],
              labels=["I", "S", "R"],
              colors=["red", "yellow", "lightgreen"])
plt.xlabel("Tijd")
plt.ylabel("Aantal personen")
plt.legend(loc=0)

plt.plot(oplossing.t, oplossing.y[1], color="purple")  # I 
plt.plot(oplossing.t, oplossing.y[2], color="green")   # R
plt.plot(oplossing.t, oplossing.y[0], color="orange")  # S

plt.show()

> **Oefening 4**: Bereken de waarde van $S$ voor het tijdstip waarop $I$ maximaal is. 

> **Oefening 5**: Hoe verandert de grafiek als beta met een kwart verminderd wordt? 

## Sociale netwerken

Het standaard SIR-model maakt de onrealistische veronderstelling dat twee willekeurige individuen telkens dezelfde kans hebben om met elkaar in contact te komen en zo mogelijks een ziekte door te geven. In werkelijkheid gaat natuurlijk niet iedereen met dezelfde mensen om. We hebben allemaal mensen waar we meer mee omgaan (meer in contact mee komen) dan met anderen. **Het geheel van wie met wie in contact staat, wordt een *sociaal netwerk* genoemd (denk aan Facebook).** De structuur van zo'n netwerk zal een sterke invloed hebben op de dynamiek van de ziekteverspreiding. 

In deze sectie zullen we bekijken hoe we een netwerk wiskundig kunnen beschrijven.


### Een voorbeeld: netwerken en grafen

In Figuur 1 zie je een voorbeeld van enkele netwerken. Elk netwerk is voorgesteld door een *graaf*. De punten vertegenwoordigen de leerlingen en worden *knopen* genoemd. De contacten tussen leerlingen worden weergegeven door lijnsegmenten tussen knopen, en worden *bogen* genoemd. **We zeggen dat twee knopen met elkaar *verbonden* zijn als ze met een boog geconnecteerd worden.** Hier gaan we ervan uit dat een knoop niet verbonden kan zijn met zichzelf. We gaan er dus van uit dat je niet met jezelf 'bevriend' kan zijn. Verder is er maar maximaal één boog mogelijk tussen twee knopen.

![Voorbeelden van gekleurde grafen die netwerken tussen kinderen van verschillende leeftijden voorstellen. De vierkantjes stellen jongens voor, de cirkels meisjes.](.image/netwerkkinderen.png)

<center>Figuur 1. Sample networks of self-reported social contacts within a classroom (Conlan et al, 2011).</center>

<center> Zoals gebruikelijk is in een genogram, een grafische voorstelling om relaties tussen mensen voor te stellen, stellen de cirkels hier meisjes voor en de vierkanten jongens. Zo zie je voor het netwerk voor kinderen tussen 4 en 5 jaar een kliekje van jongens, die bevriend zijn met een vriendengroepje van meisjes en ook nog met een ander meisje. Dat meisje is bevriend met nog een andere jongen. Er zijn ook twee meisjes die enkel met elkaar bevriend zijn en zes kinderen zonder vrienden.</center>

De figuren die men gebruikt om een netwerk of een graaf voor te stellen, zijn niet uniek. Eenzelfde netwerk kan vaak op verschillende manieren voorgesteld worden. Zoals je ziet worden de knopen voorgesteld door cirkels, vierkanten of andere vormen, die geconnecteerd zijn door lijnen, de bogen. In het vorige voorbeeld duidt de vorm van de knopen ook een geslacht in een sociaal netwerk aan; men kan dat ook doen door kleuren te gebruiken. In dat geval spreekt men van een *gekleurde graaf*.

> **Oefening 6**: Beschrijf het verschil tussen de sociale netwerken van de verschillende leeftijdsgroepen.

<div class="alert alert-box alert-info">
    Een netwerk kan voorgesteld worden door een <em>graaf</em>. De punten worden <em>knopen</em> genoemd. Knopen kunnen al dan niet met elkaar verbonden zijn. De lijnen tussen knopen worden <em>bogen</em> genoemd. We zeggen dat twee knopen met elkaar <em>verbonden</em> zijn als ze met een boog geconnecteerd worden.
</div>

Een figuur is nuttig om te bekijken hoe het netwerk eruitziet. Om er berekeningen mee te doen zijn er echter andere voorstellingen nodig. Een graaf kan wiskundig voorgesteld worden door een *verbindingsmatrix* (Engels: adjacency matrix). <br>
Als het aantal knopen in de graaf $n$ is, dan is de verbindingsmatrix een vierkante matrix V met dimensie $n \times n$. Het element $v_{ij} = 1$ als de knopen $i$ en $j$ verbonden zijn, en $v_{ij} = 0$ als ze niet verbonden zijn. <br>De verbindingsmatrix linkt grafentheorie met matrixtheorie!

Neem het onderstaande klein graafje als voorbeeld.

![Graaf met vijf knopen.](.image/graph.png)
<center> Figuur 2.</center>
Dit heeft als verbindingsmatrix

$$
V = \begin{bmatrix}
       0 & 1 & 0 & 1 & 0 \\
       1 & 0 & 1 & 0 & 1 \\
       0 & 1 & 0 & 1 & 1 \\
       1 & 0 & 1 & 0 & 1 \\
       0 & 1 & 1 & 1 & 0
     \end{bmatrix}.
$$

(A: 0, B: 1, C: 2, D: 3, E: 4. Knoop 0 (A) is niet verbonden met knoop 2 (C). Dus $v_{02} = 0$. Knoop 1 (B) is wel verbonden met knoop 2 (C). Dus $v_{12} = 1$.)<br>
Merk op dat een verbindingsmatrix altijd vierkant en altijd symmetrisch is.

In Python stellen we de matrix voor met een *NumPy array*.

In [ ]:
V = np.array([[0, 1, 0, 1, 0],  # A: eerste rij en eerste kolom
              [1, 0, 1, 0, 1],  # B: tweede rij en tweede kolom
              [0, 1, 0, 1, 1],  # C
              [1, 0, 1, 0, 1],  # D
              [0, 1, 1, 1, 0]]) # E

## Een ziekte-uitbraak in een sociaal netwerk

Het sociaal netwerk dat we beschouwen, wordt weergegeven in onderstaande figuur. De knopen, hier personen, zijn genummerd om ze makkelijk te kunnen identificeren. We houden geen rekening met geslacht of andere attributen. We zullen in dit sociaal netwerk een ziekte-uitbraak simuleren!

![Een sociaal netwerk tussen vijftien personen.](.image/socialnetwerk.png)
<center> Figuur 3.</center>

> **Oefening 7**: Voltooi de verbindingsmatrix voor het sociale netwerk.

In [ ]:
V = np.array([[0, 0, 1, ...],
              [0, 0, 1, ...],
              ...])

Laat ons nu kijken hoe we het SIR-ziekteverspreidingsmodel kunnen vertalen naar de taal van netwerken. <br> 
Aan de hand van een algemeen netwerk zullen we een veel realistischer model opstellen. Geen continue benadering meer! Dit model sluit verrasssend beter aan bij de werkelijkheid, en bovendien is het ook veel eenvoudiger om te bevatten en te simuleren. We kunnen een exacte oplossing bekomen zonder dat we afgeleiden of andere geavanceerde wiskundige technieken nodig hebben!

### Ziektedynamiek op een netwerk

In plaats van het aantal $S$-, $I$- en $R$-individuen doorheen de tijd bij te houden zoals bij het standaard SIR-model, zullen we voor elke knoop in het netwerk zijn of haar toestand bijhouden. De tijd zal niet continu variëren maar zal nu in discrete stappen voorbij gaan: $t = 0, 1, 2, 3, \ldots$. De toestand van knoop nummer $i$ op tijdstip $t$ wordt beschreven door $N_i^t\in \{S, I, R\}$. Dit wil zeggen dat knoop $i$ op tijdstip $t$ de toestand $S$ (vatbaar), $I$ (geïnfecteerd) of $R$ (resistent) kan hebben. De verandering in toestand van de knopen beschrijven we aan de hand van enkele eenvoudige regels. Analoog aan het oorspronkelijke SIR-model dat twee parameters heeft, beta (het infectiepercentage) en gamma (het herstelpercentage), heeft ook het netwerk SIR-model twee parameters.


#### Vatbare en geïnfecteerde mensen

Laten we ons eerst beperken tot vatbare en geïnfecteerde individuen. We gaan ervan uit dat vatbare individuen geïnfecteerd kunnen worden, en geïnfecteerde individuen kunnen resistent worden. Er is dus geen mogelijke overgang van geïnfecteerd naar vatbaar en ook niet van vatbaar naar resistent. Beschouw volgende regels:

1. Indien een knoop op tijdstip $t$ in toestand $S$ zit, dan heeft elke **geïnfecteerde** buur een kans $p_\text{inf}$ om de ziekte door te geven. De knoop gaat naar toestand $I$ indien minstens één buur de ziekte doorgeeft.
2. Indien een knoop op tijdstip $t$ in toestand $I$ zit, dan gaat deze naar de toestand $R$ met een kans $p_\text{res}$.

Dus, stel dat een knoop in toestand $S$ zit, en ze heeft $k$ buren die in toestand $I$ zitten. De kans dat geen enkele buur de ziekte doorgeeft, is dan:

$$
(1-p_\text{inf})^k,
$$

dus de kans dat de ziekte wel doorgegeven wordt, en er dus een transitie van toestand $S$ naar $I$ plaatsvindt, is:

$$
1 - (1-p_\text{inf})^k\,.
$$

We maakten hier gebruik van de productregel en de complementregel uit de kansrekening. 



#### Voorbeeld
Beschouw de knoop in het blauw omlijnd in het onderstaand voorbeeld. Stel dat $p_\text{inf}=0.2$, wat is dan de kans dat één van de drie zieke buren de ziekte doorgeeft?

![](.image/ziekteverspr.png)
<center> Figuur 4.</center>

Dit berekenen we met de volgende code:

In [ ]:
p_inf = 0.2
k = 3

p_ziekte_doorgegeven = 1 - (1 - p_inf)**k

print("kans om de ziekte te krijgen is", p_ziekte_doorgegeven)

Het effectief doorgeven van de ziekte kunnen we simuleren met NumPy, waar `np.random.rand()` een willekeurig getal, uniform verdeeld tussen 0 en 1, genereert. <br>We doen dat met de code in de volgende code-cel. Voer voor de simulatie die cel enkele keren uit.

In [ ]:
p_ziekte_doorgegeven > np.random.rand()

Bij `True` wordt de ziekte effectief doorgegeven, bij `False` niet. Merk op dat er een toevalsfactor in de simulatie is ingebouwd. 

> **Oefening 8**: Stel dat $p_\text{inf}=1$ (iedereen die ziek is, geeft direct de ziekte door aan al zijn of haar buren in het netwerk). Initieel zijn enkel knopen 1 en 11 geïnfecteerd in het voorbeeldnetwerk uit Figuur 3.<br> 
-  Wie is allemaal geïnfecteerd in de volgende stap? 
-  En in de stap daarna?

### Implementatie
We kunnen het model eenvoudig implementeren in Python m.b.v. SciPy. <br>Eerst zullen we een simpel sociaal netwerk genereren om dit model te illustreren:
-  We genereren daarvoor een populatie van `n` personen. Om het visueel te houden worden deze voorgesteld als punten in het $x,y$-vlak.
- Nadien genereren we een verbindingsmatrix die weergeeft of er een verbinding is tussen de knopen. 

#### Eerst zullen we de knopen van het netwerk genereren. We genereren terzelfdertijd de afstand tussen de knopen.

In [ ]:
def genereer_populatie(n):
    """Genereren van punten en bepalen van hun onderlinge afstand."""
    # n punten genereren, uniform in het xy-vlak
    X = np.random.rand(n, 2)
    # alle paarsgewijze afstanden tussen n punten
    D = distance_matrix(X, X)
    return X, D

In [ ]:
# populatie van netwerk van 200 punten genereren
n = 200
X, D = genereer_populatie(n)

In [ ]:
print(X,D)

De afstanden tussen twee personen vormen samen de afstandsmatrix $D$.

In [ ]:
# X bestaat uit 200 koppels en D is 200x200-matrix
print(X.shape, D.shape)

#### Nu zullen we de verbindingsmatrix V genereren.

Om een simpel model voor de verbindingsmatrix V te bekomen, laat ons zeggen dat de kans dat $v_{ij}=1$, dus dat knopen $i$ en $j$ verbonden zijn, gegeven wordt door:

$$
p_{ij} = \exp(-\alpha \, d_{ij})\,.
$$

**Hier geldt dat de kans op een verbinding tussen knopen $i$ en $j$ afneemt naarmate de afstand tussen de twee knopen toeneemt.** <br>
$\alpha$ is een parameter ($\alpha \geq 0$) die dit verband regelt. Een grote waarde van $\alpha$ zorgt ervoor dat twee ver uiteen gelegen knopen een heel kleine kans hebben om in verbinding te staan. Voor een kleine waarde van $\alpha$ is dit wel nog mogelijk. Bovendien geldt dat hoe groter de afstand is tussen twee knopen, hoe kleiner de kans op een verbinding. 

In [ ]:
# illustratie van effect van waarde van alpha
plt.figure() 

xwaarden = np.linspace(0, 10, 100)
plt.plot(xwaarden, np.exp(-0.1 * xwaarden), label=r"$\alpha=0.1$")       # r in omschrijving label omwille van LaTeX-code
plt.plot(xwaarden, np.exp(-0.5 * xwaarden), label=r"$\alpha=0.5$")
plt.plot(xwaarden, np.exp(-1 * xwaarden), label=r"$\alpha=1$")
plt.plot(xwaarden, np.exp(-5 * xwaarden), label=r"$\alpha=5$")
plt.plot(xwaarden, np.exp(-10 * xwaarden), label=r"$\alpha=10$")
plt.xlabel(r"Afstand $d_{ij}$")                 
plt.ylabel(r"Kans op verbinding $v_{ij}$")
plt.legend(loc=0)

plt.show()

> **Oefening 9**: Denk goed na over de betekenis van $\alpha$. Wat als $\alpha=0$? Wat als $\alpha$ heel groot is?

In [ ]:
def sample_verbindingsmatrix(D, alpha=1.0):
    """Genereren van verbindingsmatrix afhankelijk van de afstandsmatrix en alpha."""
   
    # verbindingsmatrix heeft dezelfde dimensie als de afstandsmatrix, beide zijn vierkant
    n = D.shape[1]             # aantal kolommen in D is gelijk aan de populatiegrootte
    
    # matrix aanmaken met 0 en 1 om verbindingen voor te stellen
    # alle elementen op de diagonaal zijn nul, matrix is symmetrisch
    A = np.zeros((n, n))
    
    for i in range(n):
        for j in range(i+1, n):
                 # kans op een verbinding
                 p = np.exp(- alpha * D[i,j])
                 # met een kans van p, maak een verbinding tussen i en j
                 if p > np.random.rand():
                        A[i,j] = 1
                        A[j,i] = 1      # symmetrische matrix
    return A


In [ ]:
# verbindingsmatrix van netwerk genereren voor alpha = 10
V = sample_verbindingsmatrix(D, alpha=10)
print(V)        # elke matrix kan gebruikt worden als representatie van een figuur  
print(V.min(), V.max())

In [ ]:
# visualiseren dat V uit nullen en enen bestaat
plt.imshow(V, cmap="gray")   # elke matrix kan gebruikt worden als representatie voor afbeelding, 0 zwart, 1 wit 

#### Het netwerk voorstellen met een graaf. 

Hiervoor schrijven we een nieuwe functie in Python.<br> Geïnfecteerde personen zullen weergegeven worden in het rood, resistente in het groen en vatbare in het geel. We zullen dus een gekleurde graaf gebruiken. <br>
Als de toestand van de knopen nog niet is meegegeven, kleuren we ze blauw.

Bij de lijst punten (knopen) van het netwerk, hoort dus ook een lijst van toestanden, waarbij de eerste toestand overeenkomt met de eerste knoop, de tweede toestand met de tweede knoop, enz. 

In [ ]:
 def plot_netwerk(X, V, toestanden=None):
    """Graaf van het netwerk.""" 
    n = V.shape[1]          # populatiegrootte is gelijk aan aantal kolommen van V
    
    # van elke knoop kleur nagaan en lijst van maken
    if toestanden is None:
        # geen toestanden gegeven, alle noden zijn blauw
        knoop_kleuren = "blue"
    else:
        kleur_map = {"S" : "yellow", "I" : "red", "R" : "green"}    # dictionary
        knoop_kleuren = [kleur_map[toestand] for toestand in toestanden]
    
    
    plt.figure()
    
    plt.axis("off")  # bij graaf geen assen  
    
    # plot n knopen, eerste kolom van X bevat x-coördinaat, tweede kolom y-coördinaat in juiste kleur
    plt.scatter(X[:,0], X[:,1], color=knoop_kleuren, zorder=1)    # zorder=1: punten op bovenste layer van graaf
    
    # teken de verbindingen in het grijs
    # n is de populatiegrootte en V[i,j] is de waarde van de verbinding (0 of 1)
    # als V[i,j] = 1, dan lijnstuk tussen i-de en j-de knoop
    # dus van X i-de en j-de rij nodig, dus X[i,j] nodig met x'n in eerste kolom daarvan en y's in tweede
    for i in range(n):
        for j in range(i+1, n):
            if V[i,j] == 1:
                plt.plot(X[[i,j],0], X[[i,j],1], alpha=0.8, color="grey", zorder=0)    # zorder=0: lijnen onderste layer van graaf
    plt.scatter([], [], color="yellow", label="S")       # lege punten om labels al te kunnen tonen
    plt.scatter([], [], color="red", label="I")
    plt.scatter([], [], color="green", label="R")
    plt.legend(loc=0)
    
    plt.show()

In [ ]:
plot_netwerk(X, V)       # knopen en verbindingen van ons netwerk plotten, nog zonder toestanden

#### Laat ons nu aan elk van de knopen een initiële toestand toekennen.

Initieel is iedereen in toestand $S$, behalve vijf willekeurige personen die geïnfecteerd zijn.

In [ ]:
n_inf = 5  # initieel aantal geïnfecteerden

#lijst maken van initiële toestanden 
initiele_toestanden = ["S"] * n         # lijst maken van 200 S'n
initiele_toestanden[0:n_inf] = ["I"] * n_inf  # 5 S'n vervangen dootr I, maakt niet uit welke

In [ ]:
print(initiele_toestanden)
print(len(initiele_toestanden))

In [ ]:
plot_netwerk(X, V, initiele_toestanden)     # knopen en verbindingen van ons netwerk plotten, nu met initiële toestanden

#### Overgang van ene toestand naar andere

We hebben dus een functie nodig die telkens de toestand op tijdstip $t$ omzet naar de toestand op tijdstip $t+1$. Dit is een vrij ingewikkelde functie! De overgang tussen tijdstippen noemen we een *tijdstap*.

In [ ]:
def update_toestand(toestanden, V, p_inf=1, p_res=0):
    "Functie die toestand aanpast naar nieuwe toestand per tijdstap."
    n = len(toestanden)        # aantal toestanden is populatiegrootte
    nieuwe_toestanden = []     # maak lijst om de nieuwe toestanden in op te slaan
    
    for i, toestand in enumerate(toestanden):         # ga lijst toestanden af en houd overeenkomstige index bij
        if toestand == "S":                           # persoon i is vatbaar
            # tel aantal geïnfecteerden die persoon i kent
            n_inf_kennissen = 0
            for j in range(n):
                if V[i,j] == 1 and toestanden[j] == "I":     # als persoon i in contact met geïnfecteerde persoon
                    n_inf_kennissen += 1
            # kans dat persoon i ziek wordt door een zieke kennis
            p_ziekte = 1 - (1 - p_inf)**n_inf_kennissen
            # effectief besmet of niet
            if (p_ziekte > np.random.rand()):
                toestand = "I" 
            else:
                toestand = "S"
            nieuwe_toestanden.append(toestand)
        elif toestand == "I":                          # persoon i is vatbaar
            # persoon die geïnfecteerd is, kan resistent worden
            # effectief besmet of niet
            if (p_res > np.random.rand()):
                toestand = "R"  
            else:
                toestand = "I"
            nieuwe_toestanden.append(toestand)
        elif toestand == "R":                          # persoon i is resistent
            # resistente personen blijven resistent
            nieuwe_toestanden.append("R")
    
    return nieuwe_toestanden

In [ ]:
# initiële toestanden updaten voor bepaalde p_inf en p_res voor één tijdstap
p_inf = 0.1
p_res = 0.01

nieuwe_toestanden = update_toestand(initiele_toestanden, V, p_inf, p_res)

print("aantal infecties op t=0:", 5)
print("aantal infecties op t=1:", nieuwe_toestanden.count("I"))

In [ ]:
plot_netwerk(X, V, nieuwe_toestanden)         # knopen en verbindingen van ons netwerk plotten, nu met toestanden op t=1

#### Simulatie evolutie toestanden

We kunnen dit herhalen voor een hele reeks tijdstappen aan de hand van een for-lus:

In [ ]:
def simuleer_epidemie(init_toestanden, V, tijdstappen, p_inf=1, p_res=0):
    """Simulatie van evolutie toestanden."""
    # sla de toestanden op in een lijst van lijsten
    toestanden_lijst = [init_toestanden]     # lijst huidige toestanden wordt als eerste element in toestanden_lijst gestopt
    toestanden = init_toestanden
    for t in range(tijdstappen):
        toestanden = update_toestand(toestanden, V, p_inf, p_res)
        toestanden_lijst.append(toestanden)
    return toestanden_lijst

Laat ons dit eens doen voor 100 tijdstappen.

In [ ]:
# simulatie van evolutie toestanden van initiële toestand over 100 tijdstappen
simulatie = simuleer_epidemie(initiele_toestanden, V, 100, p_inf, p_res)   # nog steeds p_inf = 0.1 en p_res = 0.01

Laat ons nu eens naar enkele snapshots doorheen de tijd kijken.

In [ ]:
# verloop na 0, 10, 20, 50, 70 en 100 tijdstappen
for t in [0, 10, 20, 50, 70, 100]:
    toestanden = simulatie[t]             # simulatie is lijst van toestanden van toestanden
    print("tijdstip {}: {} geïnfecteerd, {} resistent".format(t, toestanden.count("I"), toestanden.count("R")))
    plot_netwerk(X, V, toestanden)
    

We kunnen de voortgang makkelijker opvolgen aan de hand van een grafiek. Laat ons eens kijken hoe de verhoudingen tussen vatbaren, geïnfecteerden en resistenten wijzigen doorheen de tijd:

In [ ]:
def plot_progressiekrommen(toestanden_lijst):
    """Evolutie cijfers."""
    tijdstappen = len(toestanden_lijst)     # aantal elementen in toestanden_lijst is gelijk aan aantal tijdstappen
    # tel het aantal personen voor elke toestand per tijdstap
    S = [toestanden.count("S") for toestanden in toestanden_lijst]
    I = [toestanden.count("I") for toestanden in toestanden_lijst]
    R = [toestanden.count("R") for toestanden in toestanden_lijst]
    
    plt.figure()
    
    plt.plot(range(tijdstappen), I, color="purple", label="I")
    plt.plot(range(tijdstappen), S, color="orange", label="S")
    plt.plot(range(tijdstappen), R, color="green", label="R")
    plt.legend(loc=0)
    plt.xlabel("Tijd")
    plt.ylabel("Aantal personen")
    
    plt.show()

In [ ]:
def plot_progressievlakken(toestanden_lijst):
    """Evolutie cijfers."""
    tijdstappen = len(toestanden_lijst)     # aantal elementen in toestanden_lijst is gelijk aan aantal tijdstappen
    # tel het aantal personen voor elke toestand per tijdstap
    S = [toestanden.count("S") for toestanden in toestanden_lijst]
    I = [toestanden.count("I") for toestanden in toestanden_lijst]
    R = [toestanden.count("R") for toestanden in toestanden_lijst]
    
    plt.figure()
    
    plt.stackplot(range(tijdstappen), I, S, R,
                    labels=["I", "S", "R"], colors=["red", "yellow", "lightgreen"])
    plt.legend(loc=0)
    plt.xlabel("Tijd")
    plt.ylabel("Aantal personen")
    
    plt.show()

In [ ]:
plot_progressiekrommen(simulatie)

In [ ]:
plot_progressievlakken(simulatie)

> **Oefening 10**: Indien er te snel te veel mensen ziek worden, kan het gezondheidsapparaat overrompeld worden, met catastrofale gevolgen! Om dit te vermijden wordt het principe van *social distancing* toegepast: mensen moeten sociaal contact zo veel mogelijk vermijden. Dit zorgt ervoor dat de ziekte trager wordt doorgegeven. 
- Je kan social distancing simuleren door $\alpha$ hoger te zetten, bv. op 25. Doe dit. Zie je waarom het resultaat *flatten-the-curve*-effect noemt?
- Had je dit ook al opgemerkt bij oefening 5?

<div class="alert alert-box alert-info">
    Wil je deze notebook downloaden, maar is het bestand te groot geworden door de grafieken?<br>
    Verwijder dan eerst de output van de cellen door in het menu <b>Cell > All output > Clear</b> te kiezen.
    Je kan de notebook ook opslaan als pdf of uitprinten, net zoals je met een webpagina zou doen.
</div>

### Referenties

Conlan, A. J. K., Eames, K. T. D., Gage, J. A., von Kirchbach, J. C., Ross,J. V., Saenz, R. A.,& Gog J. R. 2011. <br> &nbsp; &nbsp; &nbsp; &nbsp; Measuring social networks in British primary schools through scientific engagement. *Proceedings of the Royal Society B, 278*(1711), 1467–1475.<br> &nbsp; &nbsp; &nbsp; &nbsp;  https://doi.org/10.1098/rspb.2010.1807

<img src=".image/cclic.png" alt="Banner" align="left" style="width:100px;"/><br><br>
Deze notebook van M. Stock voor Dwengo vzw is in licentie gegeven volgens een <a href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Naamsvermelding-NietCommercieel-GelijkDelen 4.0 Internationaal-licentie</a>. 

![Dwengo vzw](.image/logodwengo.png)